In [17]:
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser, JsonKeyOutputFunctionsParser, PydanticOutputFunctionsParser, PydanticAttrOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_community.utils.openai_functions import convert_pydantic_to_openai_function

##### JsonOutputFunctionsParser

In [8]:
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

openai_functions = [convert_pydantic_to_openai_function(Joke)]

In [10]:
llm = ChatOpenAI().bind_functions(openai_functions)

prompt = ChatPromptTemplate.from_messages([
                                            ("system","You are a helpful assistant."),
                                            ("human","{user_query}")
                                        ])

parser = JsonOutputFunctionsParser()

chain = prompt | llm | parser

output = chain.invoke({"user_query":"Tell me a joke."})

output

{'setup': "Why couldn't the bicycle find its way home?",
 'punchline': 'Because it lost its bearings!'}

##### JsonKeyOutputFunctionsParser

In [11]:
# Example 1 
parser = JsonKeyOutputFunctionsParser(key_name="setup")

chain = prompt | llm | parser

output = chain.invoke({"user_query":"Tell me a joke."})

output

"Why couldn't the bicycle stand up by itself?"

In [18]:
# Example 2
from typing import List

class Jokes(BaseModel):
    jokes : List[Joke]
    funniness_level : int

openai_functions = [convert_pydantic_to_openai_function(Jokes)]

llm = ChatOpenAI().bind_functions(openai_functions)

parser = JsonKeyOutputFunctionsParser(key_name="jokes")

chain = prompt | llm | parser

output = chain.invoke({"user_query":"Tell me 2 jokes."})

output


[{'setup': "Why couldn't the bicycle find its way home?",
  'punchline': 'It lost its bearings!'},
 {'setup': 'What do you call a fake noodle?', 'punchline': 'An impasta!'}]

##### PydanticOutputFunctionsParser

In [20]:
class Joke(BaseModel):
    setup: str = Field(description="Setup of a joke.")
    punchline :str = Field(description="Punchline of a joke.")

    @validator("setup")
    def question_ends_with_question_mark(field):
        if field[-1] != "?":
            raise ValueError("Invalid Input, setup must end with a question mark.")
        return field
    

parser = PydanticOutputFunctionsParser(pydantic_schema=Joke)

openai_functions = [convert_pydantic_to_openai_function(Joke)]

model = ChatOpenAI().bind_functions(openai_functions)

chain = prompt | model | parser

output = chain.invoke({"user_query":"Tell me a joke"})

In [21]:
output

Joke(setup='Why did the scarecrow win an award?', punchline='Because he was outstanding in his field!')

##### PydanticAttrOutputFunctionsParser


In [27]:
parser = PydanticAttrOutputFunctionsParser(pydantic_schema=Joke,attr_name="punchline")

openai_functions=[convert_pydantic_to_openai_function(Joke)]

llm = ChatOpenAI().bind_functions(openai_functions)

chain = prompt | llm | parser

output = chain.invoke({"user_query":"tell me a joke"})

output

'Because he was outstanding in his field!'